[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2022CSharp/main?urlpath=lab%2Ftree%2FDay07.ipynb)

<h2>--- Day 7: No Space Left On Device ---</h2><p>You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?</p>
<p>The device the Elves gave you has problems with more than just its communication system. You try to run a system update:</p>
<pre><code>$ system-update --please --pretty-please-with-sugar-on-top
<span title="E099 PROGRAMMER IS OVERLY POLITE">Error</span>: No space left on device
</code></pre>
<p>Perhaps you can delete some files to make space for the update?</p>
<p>You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:</p>
<pre><code>$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
</code></pre>
<p>The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called <code>/</code>. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.</p>
<p>Within the terminal output, lines that begin with <code>$</code> are <em>commands you executed</em>, very much like some modern computers:</p>
<ul>
<li><code>cd</code> means <em>change directory</em>. This changes which directory is the current directory, but the specific result depends on the argument:
  <ul>
  <li><code>cd x</code> moves <em>in</em> one level: it looks in the current directory for the directory named <code>x</code> and makes it the current directory.</li>
  <li><code>cd ..</code> moves <em>out</em> one level: it finds the directory that contains the current directory, then makes that directory the current directory.</li>
  <li><code>cd /</code> switches the current directory to the outermost directory, <code>/</code>.</li>
  </ul>
</li>
<li><code>ls</code> means <em>list</em>. It prints out all of the files and directories immediately contained by the current directory:
  <ul>
  <li><code>123 abc</code> means that the current directory contains a file named <code>abc</code> with size <code>123</code>.</li>
  <li><code>dir xyz</code> means that the current directory contains a directory named <code>xyz</code>.</li>
  </ul>
</li>
</ul>
<p>Given the commands and output in the example above, you can determine that the filesystem looks visually like this:</p>
<pre><code>- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
</code></pre>
<p>Here, there are four directories: <code>/</code> (the outermost directory), <code>a</code> and <code>d</code> (which are in <code>/</code>), and <code>e</code> (which is in <code>a</code>). These directories also contain files of various sizes.</p>
<p>Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the <em>total size</em> of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)</p>
<p>The total sizes of the directories above can be found as follows:</p>
<ul>
<li>The total size of directory <code>e</code> is <em>584</em> because it contains a single file <code>i</code> of size 584 and no other directories.</li>
<li>The directory <code>a</code> has total size <em>94853</em> because it contains files <code>f</code> (size 29116), <code>g</code> (size 2557), and <code>h.lst</code> (size 62596), plus file <code>i</code> indirectly (<code>a</code> contains <code>e</code> which contains <code>i</code>).</li>
<li>Directory <code>d</code> has total size <em>24933642</em>.</li>
<li>As the outermost directory, <code>/</code> contains every file. Its total size is <em>48381165</em>, the sum of the size of every file.</li>
</ul>
<p>To begin, find all of the directories with a total size of <em>at most 100000</em>, then calculate the sum of their total sizes. In the example above, these directories are <code>a</code> and <code>e</code>; the sum of their total sizes is <code><em>95437</em></code> (94853 + 584). (As in this example, this process can count files more than once!)</p>
<p>Find all of the directories with a total size of at most 100000. <em>What is the sum of the total sizes of those directories?</em></p>

In [8]:
using System.IO;

In [9]:
var input = System.IO.File.ReadAllLines("input/07.txt");

In [10]:
record Child(string Name);
record File(string Name, long Size) : Child(Name);
record Directory(
    string Name, 
    Directory Parent, 
    Dictionary<string, Child> Children) : Child(Name);

In [11]:
void parseTerminalOutput(Directory root, string[] input)
{
    var cd = root;
    foreach (var line in input)
    {
        if (line.StartsWith(@"$ cd /"))
        {
            cd = root;
        }
        else if (line.StartsWith(@"$ cd .."))
        {
            cd = cd.Parent;
        }
        else if (line.StartsWith(@"$ cd "))
        {
            var name = line.Split(" ").Last().Trim();
            cd = (Directory)cd.Children[name];
        }
        else if (line.StartsWith("dir"))
        {
            var name = line.Split(" ").Last().Trim();
            cd.Children[name] = new Directory(name, cd, new Dictionary<string, Child>());
        }
        else if (Char.IsNumber(line[0]))
        {
            var split = line.Split(" ");
            var size = long.Parse(split.First().Trim());
            var name = split.Last().Trim();
            cd.Children[name] = new File(name, size);
        }
    }
}

In [12]:
void calcDirectorySizes(Directory dir, Dictionary<Directory, long> sizes)
{
    long size = 0;
    foreach (var child in dir.Children)
    {
        if (child.Value is File f)
        {
            size += f.Size;
        }
        else if (child.Value is Directory d)
        {
            calcDirectorySizes(d, sizes);
            size += sizes[d];
        }
    }
    sizes[dir] = size;
}

In [13]:
#!time
var root = new Directory("/", null, new Dictionary<string, Child>());
parseTerminalOutput(root, input);

var sizes = new Dictionary<Directory, long>();
calcDirectorySizes(root, sizes);

sizes.Values
    .Where(size => size <= 100_000)
    .Sum()

1350966

Wall time: 168.3076ms

<h2 id="part2">--- Part Two ---</h2><p>Now, you're ready to choose a directory to delete.</p>
<p>The total disk space available to the filesystem is <code><em>70000000</em></code>. To run the update, you need unused space of at least <code><em>30000000</em></code>. You need to find a directory you can delete that will <em>free up enough space</em> to run the update.</p>
<p>In the example above, the total size of the outermost directory (and thus the total amount of used space) is <code>48381165</code>; this means that the size of the <em>unused</em> space must currently be <code>21618835</code>, which isn't quite the <code>30000000</code> required by the update. Therefore, the update still requires a directory with total size of at least <code>8381165</code> to be deleted before it can run.</p>
<p>To achieve this, you have the following options:</p>
<ul>
<li>Delete directory <code>e</code>, which would increase unused space by <code>584</code>.</li>
<li>Delete directory <code>a</code>, which would increase unused space by <code>94853</code>.</li>
<li>Delete directory <code>d</code>, which would increase unused space by <code>24933642</code>.</li>
<li>Delete directory <code>/</code>, which would increase unused space by <code>48381165</code>.</li>
</ul>
<p>Directories <code>e</code> and <code>a</code> are both too small; deleting them would not free up enough space. However, directories <code>d</code> and <code>/</code> are both big enough! Between these, choose the <em>smallest</em>: <code>d</code>, increasing unused space by <code><em>24933642</em></code>.</p>
<p>Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. <em>What is the total size of that directory?</em></p>

In [14]:
#!time
sizes.Values
    .Where(size => size >= sizes[root] - 40_000_000)
    .Min()

6296435

Wall time: 84.8729ms